#COMP396 WINTER 2020: SMILES +LSTM
Description: The goal of this milestone is to compute quantum properties using a SMILES string representation with an LSTM 

Credits :
Adam Yala, Intro to Machine Learning Labs, 2019 Github repository (https://github.com/yala/introML_chem) 




## Imports

In [0]:
import re
import sys

import math
import numpy as np
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [34]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
root = "/content/drive/My Drive/dsb2/"

In [36]:
import os

file_list = os.listdir(root)

num_mols = len(file_list)
print(num_mols)

13000


# Input retrieval 


In [0]:
def read_xyz(file_name):
    with open(file_name, 'rb') as file:
        num_atoms = int(file.readline())
        properties = file.readline().split()[5:17] # only take the properties used in the experiments 
        properties = [num.replace(b'*^', b'e') for num in properties] 
        properties = [float(prop) for prop in properties]
        atom_types = [0]*num_atoms
        coords = np.array(np.zeros([num_atoms,3]))
        for na in range(num_atoms):
            coord_line = file.readline().split()
            atom_types[na] = coord_line[0]
            xyz_coords = coord_line[1:4]
            xyz_coords = [num.replace(b'*^', b'e') for num in xyz_coords] 
            coords[na,:] = [float(num) for num in xyz_coords]  
        vib_freqs = file.readline()
        smiles = file.readline().split()[0]
        inchis = file.readline()
        
    return smiles, properties, atom_types, coords

In [0]:
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [39]:
import multiprocessing as mp
import numpy as np
from multiprocessing.pool import ThreadPool
import time


start = time.time()
N= mp.cpu_count()
files = os.scandir(root)
print (N)
with mp.pool.ThreadPool(processes = 8) as p:
        results= p.map(read_xyz, [root+file.name for file in files])
      
end = time.time()
print(end - start)

2
15.608777046203613


In [40]:
print(len(results))

13000


In [0]:
def column(matrix, i):
    return [row[i] for row in matrix]

smiles =column(results,0)
properties =column(results,1)
atom_types =column(results,2)

In [0]:
import pandas as pd
import numpy as np

In [43]:
dtype = [('Col1','int32'), ('Col2','float32'), ('Col3','float32'),('Col4','int32'), ('Col5','float32'), ('Col6','float32'),('Col7','int32'), ('Col8','float32'), ('Col9','float32'),('Col10','int32'), ('Col11','float32'), ('Col12','float32'),('Col13','float32'), ('Col14','float32'),('Col15','float32'), ('Col16','float32'),('Col17','float32')]
values = properties
index = ['Row'+str(i) for i in range(1, len(values)+1)]

df = pd.DataFrame(values, index=index)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,0.3676,68.74,-0.2352,0.0065,0.2417,814.7537,0.142229,-363.839405,-363.831976,-363.831032,-363.870471,29.280
Row2,1.8866,64.52,-0.2485,-0.0075,0.2409,794.2784,0.129457,-383.721976,-383.714630,-383.713685,-383.753023,28.544
Row3,1.7780,73.30,-0.2542,0.0838,0.3380,883.2429,0.177401,-349.011613,-349.003832,-349.002888,-349.042983,30.929
Row4,1.2074,73.61,-0.2583,0.0729,0.3312,881.9664,0.177890,-349.011971,-349.004251,-349.003307,-349.043189,31.339
Row5,2.1403,66.86,-0.2510,0.0868,0.3378,831.9533,0.152815,-384.933448,-384.925779,-384.924835,-384.964883,29.600


In [44]:
len(df)

13000

In [0]:
std=df[0].std()

##Standardizing target properties

In [46]:
# Normalize target values to have a mean of 0 and std of 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
data_scaled=scaler.fit_transform(df)
df = pd.DataFrame(data_scaled , index=index)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,-1.497174,0.376456,0.381924,-0.143865,-0.337901,-0.570023,0.300064,-0.077239,-0.077249,-0.077249,-0.077207,0.189786
Row2,-0.487877,-0.094090,-0.147182,-0.438433,-0.354346,-0.652968,-0.079868,-0.534428,-0.534445,-0.534445,-0.534387,0.027847
Row3,-0.560036,0.884913,-0.373942,1.482575,1.641695,-0.292575,1.346334,0.263719,0.263720,0.263720,0.263736,0.552609
Row4,-0.939170,0.919479,-0.537050,1.253233,1.501910,-0.297746,1.360880,0.263710,0.263710,0.263710,0.263732,0.642819
Row5,-0.319307,0.166829,-0.246638,1.545697,1.637584,-0.500348,0.614968,-0.562285,-0.562295,-0.562295,-0.562253,0.260194


## Data Processing

In [47]:

def get_data(smiles_array,target_array,start, end):
   # Get smiles and targets
    smiles, Y = [], []
    for i in range(start,end):
      smiles.append(smiles_array[i].decode("utf-8"))
      Y.append(target_array[i])
    
    return smiles, Y

trainSmiles, trainY = get_data(smiles, df[0],2000,13000)
devSmiles, devY = get_data(smiles, df[0], 1000,2000)
testSmiles, testY = get_data(smiles, df[0], 0,1000)

allSmiles = trainSmiles + devSmiles + testSmiles

print(f'Num Train = {len(trainSmiles):,}')
print(f'Num Dev   = {len(devSmiles):,}')
print(f'Num Test  = {len(testSmiles):,}')

print(f'Example data point: smiles = {trainSmiles[0]}, mu = {trainY[0]}')

Num Train = 11,000
Num Dev   = 1,000
Num Test  = 1,000
Example data point: smiles = N=CN(CC=O)C=O, mu = -0.01611888520276374


In [48]:
allSmiles[0]

'N=CN(CC=O)C=O'

## Dataset Class

In [0]:
class QM9Dataset(Dataset):
    def __init__(self, X, Y):
      self.X, self.Y = X, Y
      assert len(X) == len(Y)

    def __len__(self):
       return len(self.X)

    def __getitem__(self, i):
      return np.array(self.X[i]), self.Y[i]

## Model and Training Settings




In [0]:
batch_size = 64
epochs = 10
lr = 1e-4
weight_decay = 1e-4
max_len = 100
embedding_size = 300
hidden_size = 300
output_size = 1  
dropout = 0.6
use_cuda = True

## Utility Functions

In [0]:
def param_count(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)
  
def mae(targets, preds):
    return mean_absolute_error(targets, preds)

## Training Procedure

In [0]:
def train_epoch(model, train_loader, optimizer, epoch, std):
    model.train()  # Set the nn.Module to train mode. 
    total_loss = 0
    total_mae = 0
    num_samples = len(train_loader.dataset)
    num_batches = 0
    for batch_idx, (data, target) in enumerate(train_loader):  # 1) get batch
        # Adjust dimensions of target and cast to float
        target = target.unsqueeze(1).float()
      
        # Move to cuda
        if next(model.parameters()).is_cuda:
            data, target = data.cuda(), target.cuda()
      
        # Reset gradient data to 0
        optimizer.zero_grad()
        
        # Get prediction for batch
        output = model(data)
        
        # 2) Compute loss
        loss = F.mse_loss(output, target)
        
        # 3) Do backprop
        loss.backward()
        
        # 4) Update model
        optimizer.step()
        
        # Do book-keeping to track rmse and avg loss
        total_loss += loss.detach()  # Don't keep computation graph 
        total_mae += mae(target.cpu().data.numpy(), output.cpu().data.numpy())
        num_batches += 1
    return (total_mae/len(train_loader.dataset))*std


## Evaluation Procedure

In [0]:
def eval_epoch(model, test_loader, std):
    model.eval()
    test_loss = 0
    test_mae = 0
    num_batches = 0
    for data, target in test_loader:
        target = target.unsqueeze(1).float()
      
        # Move to cuda
        if next(model.parameters()).is_cuda:
            data, target = data.cuda(), target.cuda()
        
        output = model(data)
        
        test_loss += F.mse_loss(output, target).item()  # sum up batch loss
        test_mae += mae(target.cpu().data.numpy(), output.cpu().data.numpy())
        num_batches += 1

    test_loss /= len(test_loader.dataset)
    test_mae /= num_batches
    return test_mae*std
    

## Define Vocab and Character-to-Index Mapping

In [54]:
# Define vocab
vocab = {char for smiles in allSmiles for char in smiles}

print(f'Vocab = {vocab}')

# Create word to index mapping
padding_idx = 0
char_to_index = {char: index + 1 for index, char in enumerate(vocab)}
vocab_size = len(char_to_index) + 1

print(f'Vocab size = {vocab_size:,}')

Vocab = {'[', '3', 'F', '4', 'O', ')', '#', '-', 'N', ']', '+', '(', '2', '1', '=', 'H', 'C'}
Vocab size = 18


In [55]:
char_to_index 

{'#': 7,
 '(': 12,
 ')': 6,
 '+': 11,
 '-': 8,
 '1': 14,
 '2': 13,
 '3': 2,
 '4': 4,
 '=': 15,
 'C': 17,
 'F': 3,
 'H': 16,
 'N': 9,
 'O': 5,
 '[': 1,
 ']': 10}

## Map Characters to Indices

In [56]:
trainX = [[char_to_index[char] for char in smiles] for smiles in trainSmiles]
devX =   [[char_to_index[char] for char in smiles] for smiles in devSmiles]
testX =  [[char_to_index[char] for char in smiles] for smiles in testSmiles]

print(f'Smiles string = {trainSmiles[0]}')
print(f'Indices of first train SMILES = {trainX[0]}')
print(f'Last five indices = {trainX[0][-5:]}')


print(f'Smiles string = {trainSmiles[2]}')
print(f'Indices of second train SMILES = {trainX[2]}')
print(f'Last five indices = {trainX[2][-5:]}')

Smiles string = N=CN(CC=O)C=O
Indices of first train SMILES = [9, 15, 17, 9, 12, 17, 17, 15, 5, 6, 17, 15, 5]
Last five indices = [5, 6, 17, 15, 5]
Smiles string = CN=C(NC=O)C#C
Indices of second train SMILES = [17, 9, 15, 17, 12, 9, 17, 15, 5, 6, 17, 7, 17]
Last five indices = [5, 6, 17, 7, 17]


## Add Padding

In [57]:
#add all elemnets the pad with os at the end so that they have the same length
trainX = [seq[:max_len] + [padding_idx] * (max_len - len(seq)) for seq in trainX]
devX =   [seq[:max_len] + [padding_idx] * (max_len - len(seq)) for seq in devX]
testX =  [seq[:max_len] + [padding_idx] * (max_len - len(seq)) for seq in testX]


print(f'Smiles string = {trainSmiles[0]}')
print(f'Indices of first train SMILES = {trainX[0]}')
print(f'Last five indices = {trainX[0][-5:]}')


print(f'Smiles string = {trainSmiles[2]}')
print(f'Indices of second train SMILES = {trainX[2]}')
print(f'Last five indices = {trainX[2][-5:]}')

Smiles string = N=CN(CC=O)C=O
Indices of first train SMILES = [9, 15, 17, 9, 12, 17, 17, 15, 5, 6, 17, 15, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Last five indices = [0, 0, 0, 0, 0]
Smiles string = CN=C(NC=O)C#C
Indices of second train SMILES = [17, 9, 15, 17, 12, 9, 17, 15, 5, 6, 17, 7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Last five indices = [0, 0, 0, 0, 0]


## Build Dataset/DataLoader

In [0]:
# Build Dataset
train = QM9Dataset(trainX, trainY)
dev = QM9Dataset(devX, devY)
test = QM9Dataset(testX, testY)

# Build DataLoader
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

# LSTM Model

In [0]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, padding_idx, embedding_size, hidden_size, output_size, dropout):
        super(LSTMModel, self).__init__()
        
        # Embedding layer
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx)
        
        # LSTM (RNN)
        self.rnn = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        
        # Fully connected layer
        self.output = nn.Linear(hidden_size, output_size)
        
        # Dropout (regularization)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):  # batch_size x seq_length
        # Embed
        embedded = self.embed(x)  # batch_size x seq_length x embedding_size
      
        # Run RNN
        o, _ = self.rnn(embedded)  # batch_size x seq_length x hidden_size
        
        # Dropout
        o = self.dropout(o)  # batch_size x seq_length x hidden_size
        
        # Max pooling across sequence
        o, _ = torch.max(o, dim=1)    # batch_size x hidden_size
        
        # Output layer
        out = self.output(o)  # batch_size x output_size
        
        return out

In [60]:
model = LSTMModel(vocab_size, padding_idx, embedding_size, hidden_size, output_size, dropout)

print(model)
print(f'Number of parameters = {param_count(model):,}')

# Move to cuda
if use_cuda and torch.cuda.is_available():
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) 

LSTMModel(
  (embed): Embedding(18, 300, padding_idx=0)
  (rnn): LSTM(300, 300, batch_first=True)
  (output): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
)
Number of parameters = 728,101


## Train SMILES +LSTM

In [62]:
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.00001)
best_val_error = None
for epoch in range(1, 51):
   # lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train_epoch(model, train_loader, optimizer, epoch,std)
    val_error = eval_epoch(model,  dev_loader,std)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = eval_epoch(model,  test_loader,std)
        best_val_error = val_error

    print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Validation MAE: {:.7f}, '
          'Test MAE: {:.7f}'.format(epoch, lr, loss, val_error, test_error))   

Epoch: 001, LR: 0.000100, Loss: 0.0152767, Validation MAE: 0.9192534, Test MAE: 0.9486582
Epoch: 002, LR: 0.000100, Loss: 0.0134828, Validation MAE: 0.8807519, Test MAE: 0.9114912
Epoch: 003, LR: 0.000100, Loss: 0.0129735, Validation MAE: 0.8734667, Test MAE: 0.9027366
Epoch: 004, LR: 0.000100, Loss: 0.0124182, Validation MAE: 0.8541857, Test MAE: 0.8701979
Epoch: 005, LR: 0.000100, Loss: 0.0119964, Validation MAE: 0.8576912, Test MAE: 0.8701979
Epoch: 006, LR: 0.000100, Loss: 0.0117619, Validation MAE: 0.8477703, Test MAE: 0.8818314
Epoch: 007, LR: 0.000100, Loss: 0.0114173, Validation MAE: 0.8291495, Test MAE: 0.8570862
Epoch: 008, LR: 0.000100, Loss: 0.0111475, Validation MAE: 0.8187288, Test MAE: 0.8464440
Epoch: 009, LR: 0.000100, Loss: 0.0109230, Validation MAE: 0.8085317, Test MAE: 0.8358742
Epoch: 010, LR: 0.000100, Loss: 0.0107744, Validation MAE: 0.8043904, Test MAE: 0.8330615
Epoch: 011, LR: 0.000100, Loss: 0.0106443, Validation MAE: 0.8079753, Test MAE: 0.8330615
Epoch: 012